This notebook scrapes the timelines <A HREF="https://en.wikipedia.org/wiki/Timeline_of_the_Barack_Obama_presidency">found on Wikipedia</A> and saves each year as a csv with fields: data, event

In [1]:
# The usual suspects...
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
import time # avoid swamping the americanrhetoric website
import os # to check if directory exists and create it if it doesn't
from datetime import date, datetime # to parse speech date

In [2]:
# Check if Data and associated folders exist for saving speech CSVs to.
# https://djangocentral.com/check-if-a-directory-exists-if-not-create-it/

MYDIRS = ["DataWiki", "USATimeline"]

# If folder doesn't exist, then create it.
for MYDIR in MYDIRS:
    CHECK_FOLDER = os.path.isdir(MYDIR)
    if CHECK_FOLDER:
        print(MYDIR, "folder already exists.")
    else:
        os.makedirs(MYDIR)
        print("created folder : ", MYDIR)

DataWiki folder already exists.
created folder :  USATimeline


In [119]:
# Barack Obama Presidential timeline
url = 'https://en.wikipedia.org/wiki/Timeline_of_the_Barack_Obama_presidency_(2011)'
links = []
for year in range(2009, 2017):
    if year==2016:
        links.append('https://en.wikipedia.org/wiki/Timeline_of_the_Barack_Obama_presidency_(2016%E2%80%932017)')
    else:
        links.append('https://en.wikipedia.org/wiki/Timeline_of_the_Barack_Obama_presidency_('+str(year)+')')
for url in links:
    try:
        page = requests.get(url, headers={'user-agent': 'Mozilla/5.0'})
    except Exception as e:
        print(e, speechlink)
        #continue # skip this page and go back to beginning of for loop
    soup = BeautifulSoup(page.content, 'html.parser')
    year = re.search('(20[0-9]+)', soup.title.text).group(0)
    months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    li_tags = [x for x in soup.find_all('li') if (soup.find_all('li') is not None) and (x.find('b') is not None)]
    tmptimeline = [y.text for y in li_tags if y.text.split(' ')[0] in months]
    timeline = [re.sub('^(.*?)[\–|−|-|—|-]\s', '', x) for x in tmptimeline]
    # regex to grab everything to left of first – : ^[^–]*
    # note – is not -  I don't think my keyboard has –
    # turns out there are three different –,−,- floating around, yeah first two are different!S
    # search finds matching text, group(0) returns all of the matching text, .rstrip() removes training space
    # re.search("^[^–]*", txt).group(0)
    dates = [ re.search("^[^–|−|-|—|-]*", txt).group(0).rstrip()+', '+ year for txt in tmptimeline]
    cleanDates = [datetime.strptime(date, '%B %d, %Y').strftime("%d/%m/%Y") for date in dates]
    # combine dates and timeline into a df and save as csv
    metadata=list(zip(cleanDates, timeline))
    df = pd.DataFrame(metadata)
    df.to_csv('DataWiki/' + 'wiki_timeline_'+year+'.csv', encoding='utf-8', header=False, index=False)
    print(year,"saved!")
    time.sleep(3)

2009 saved!
2010 saved!
2011 saved!
2012 saved!
2013 saved!
2014 saved!
2015 saved!
2016 saved!


In [22]:
# USA Timeline
url = 'https://en.wikipedia.org/wiki/2010_in_the_United_States'
links = []
for year in range(2004, 2005):
    links.append('https://en.wikipedia.org/wiki/'+str(year)+'_in_the_United_States')
for url in links:
    try:
        page = requests.get(url, headers={'user-agent': 'Mozilla/5.0'})
    except Exception as e:
        print(e, speechlink)
        #continue # skip this page and go back to beginning of for loop
    soup = BeautifulSoup(page.content, 'html.parser')
    startEvents = str(soup).find('<h2><span class="mw-headline" id="Events">Events</span>')
    year = re.search('(20[0-9]+)', soup.title.text).group(0)
    months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    li_tags = [x for x in soup.find_all('li') if (soup.find_all('li') is not None)]
    tmptimeline = [y.text for y in li_tags if y.text.split(' ')[0] in months]
    timeline = [re.sub('^(.*?)[\–|−|-|—|-|\n]\s', '', x) for x in tmptimeline]
    # regex to grab everything to left of first – : ^[^–]*
    # note – is not -  I don't think my keyboard has –
    # turns out there are three different –,−,- floating around, yeah first two are different!S
    # search finds matching text, group(0) returns all of the matching text, .rstrip() removes training space
    # re.search("^[^–]*", txt).group(0)
    dates = [ re.search("^[^–|−|-|—|-|\n]*", txt).group(0).rstrip()+', '+ year for txt in tmptimeline]
    cleanDates = [datetime.strptime(date, '%B %d, %Y').strftime("%d/%m/%Y") for date in dates]
    # combine dates and timeline into a df and save as csv
    metadata=list(zip(cleanDates, timeline))
    df = pd.DataFrame(metadata)
    df.to_csv('USATimeline/' + 'wiki_timeline_'+year+'.csv', encoding='utf-8', header=False, index=False)
    print(year,"saved!")
    time.sleep(3)

AttributeError: 'builtin_function_or_method' object has no attribute 'text'